In [1]:
import os
import sys

# change working directory
current_dir = os.getcwd() 
print(f"Current Working Directory: {current_dir}")
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")
#We need to be in the main directory that contains tests, models, etc folders

Current Working Directory: /lambda/nfs/pranjal-codebase/tests
Current Working Directory: /lambda/nfs/pranjal-codebase


In [2]:
from coarse_grain_model import GemmaWithSlidingWindow
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gc

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-27 09:14:17.062695: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-27 09:14:17.076471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753607657.093594    4120 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753607657.102133    4120 cuda_blas.cc:1407] Unable to 

In [9]:
def model_response(model_path,prompts,window_mask=None):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    print(f"Loading generative model from: {model_path}")
    tokenizer2 = AutoTokenizer.from_pretrained(model_path)
    if window_mask:
        print("Applying attention mask beyond previous"+str(window_mask)+"tokens")
        model2 = GemmaWithSlidingWindow.from_pretrained(model_path,window_mask)
    else:
        model2 = AutoModelForCausalLM.from_pretrained(model_path)
    model2.to(device)
    model2.eval()

    responses=[" "]*len(prompts)
    i=0
    for prompt in prompts:
        print("input=",prompt)
        inputs = tokenizer2(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model2.generate(
                **inputs,
                max_new_tokens=50,
                pad_token_id=tokenizer2.eos_token_id
            )
        full_response = tokenizer2.decode(outputs[0], skip_special_tokens=True)
        responses[i] = full_response[len(prompt):].strip()
        print("llm output=",responses[i])
        i+=1
    del model2
    del tokenizer2
    # 2. Force Python's garbage collection
    gc.collect()
    # 3. CRITICAL: Empty the CUDA cache
    torch.cuda.empty_cache()

    return responses

In [10]:
prompts=["Bla bla black sheep have you any",]
responses0=model_response("./models/gemma2b-it",prompts)
responses1=model_response("./models/gemma2b-it",prompts,window_mask=256)
responses2=model_response("./models/gemma2b-it_sw256/checkpoint-3750",prompts)
responses3=model_response("./models/gemma2b-it_sw256/checkpoint-3750",prompts,window_mask=256)

Using device: cuda
Loading generative model from: ./models/gemma2b-it


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.33it/s]


input= Bla bla black sheep have you any
llm output= wool?

The question is a riddle, and the answer is no.

The riddle is asking if the sheep has wool, but the answer is no. This is a trick question, as the question is asking for a negative answer.
Using device: cuda
Loading generative model from: ./models/gemma2b-it
Applying attention mask beyond previous256tokens


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.15it/s]


input= Bla bla black sheep have you any
llm output= wool?

The question is a riddle, and the answer is no.

The riddle is asking if the sheep has wool, but the answer is no. This is a trick question, as the question is asking for a negative answer.
Using device: cuda
Loading generative model from: ./models/gemma2b-it_sw256/checkpoint-3750


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


input= Bla bla black sheep have you any
llm output= wool?

Yes, yes, I have some wool,

I have some wool, I have some wool.

I have some wool, I have some wool,

I have some wool, I have some wool.

I have some wool
Using device: cuda
Loading generative model from: ./models/gemma2b-it_sw256/checkpoint-3750
Applying attention mask beyond previous256tokens


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.25it/s]


input= Bla bla black sheep have you any
llm output= wool?

Yes, yes, I have some wool,

I have some wool, I have some wool.

I have some wool, I have some wool,

I have some wool, I have some wool.

I have some wool


In [11]:
model_path="./models/gemma2b-it_sw256/checkpoint-3750"

In [13]:
"sw" in model_path

True

In [18]:
import re
model_path="./models/gemma2b-it_sw_25_2/checkpoint-3750"
match = re.search(r"sw(\d+)", model_path)

if match:
    number = match.group(1)
    print(f"Found number: {number}")
else:
    # This block runs if the pattern was not found
    raise ValueError("model path is not according to convention")

ValueError: model path is not according to convention